# Imports

In [1]:
# Imports
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import tensorflow.image as tf_image
from tensorflow.keras import layers, models, callbacks, Model

2022-12-07 00:43:07.658647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 00:43:07.658666: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize 
import nltk
from gensim.models import Word2Vec
# from sklearn.model_selection import train_test_split
import gensim.downloader as api
from keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/lscr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Paths

In [3]:
# Paths

rel_imagedir_path = '../../../data/screenshots/'
legit_image_path = os.path.join(rel_imagedir_path, 'legit_screenshots')
scam_image_path = os.path.join(rel_imagedir_path, 'scam_screenshots')

In [4]:
legit_image_path, scam_image_path

('../../../data/screenshots/legit_screenshots',
 '../../../data/screenshots/scam_screenshots')

# Loading Data

In [5]:
def load_data(key_df, legit_imagedir_path, scam_imagedir_path):
    
    X_image = []
    X_text = []
    y = []
    
    for index, row in key_df.iterrows():
        if index % 500 == 0:
            print(f'loaded {index} data points')
        
        # Append image to X_image
        if row['target'] == 0:
            image_path = os.path.join(legit_imagedir_path, f"{row['num_of_picture']}.png")
            np_legit_image = imread(image_path)[:, :, :3] # slice off the alpha channel
            np_legit_image = tf_image.resize(np_legit_image, (300, 400))
            X_image.append(np_legit_image)
            
        elif row['target'] == 1:
            image_path = os.path.join(scam_imagedir_path, f"{row['num_of_picture']}.png")
            np_legit_image = imread(image_path)[:, :, :3] # slice off the alpha channel
            np_legit_image = tf_image.resize(np_legit_image, (300, 400))
            X_image.append(np_legit_image)
        
        # Load text
        X_text.append(row['text'])
        
        y.append(row['target'])
    
    print('\nFinished loading data!')
    # print(X_image)
    return np.array(X_image), X_text, np.array(y)

In [6]:
merged_key_df = pd.read_csv('merged_key_df.csv', index_col=0)

In [7]:
X_image, X_text, y = load_data(merged_key_df, legit_image_path, scam_image_path)

loaded 0 data points


2022-12-07 00:43:10.229367: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-07 00:43:10.229384: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-07 00:43:10.229397: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lscr): /proc/driver/nvidia/version does not exist
2022-12-07 00:43:10.229859: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


loaded 500 data points
loaded 1000 data points
loaded 1500 data points
loaded 2000 data points
loaded 2500 data points
loaded 3000 data points
loaded 3500 data points
loaded 4000 data points

Finished loading data!


In [8]:
del merged_key_df

# Proproccess Text

In [9]:
# Helper Functions

def clean(text):
    text = text.split()
    words_only = [word for word in text if word.isalpha()]
    for punctuation in string.punctuation:
        words_only = [word.replace(punctuation, ' ').lower() for word in words_only] # Remove Punctuation
    # lowercased = text.lower() # Lower Case
    #tokenized = word_tokenize(lowercased) # Tokenize
    # words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    # lemma=WordNetLemmatizer() # Initiate Lemmatizer
    # lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return without_stopwords

def text_cleaner(list_text):
    print('Cleaning text')
    list_clean_text=[]
    for text in list_text:
        clean_txt=clean(text) # Use clean function
        list_clean_text.append(clean_txt)

    return list_clean_text

def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return embedded_sentence

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    print('Embedding with word2vec')
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

def preprocess_pad(list_text):
    # Clean - remove punctuation and stopwords
    list_ = text_cleaner(list_text)
    
    # Embed with word2vec
    word2vec_transfer = api.load('glove-wiki-gigaword-100')
    list_ = embedding(word2vec_transfer, list_)
    
    # Pad Sequences
    print('Padding sequences')
    list_ = pad_sequences(list_, dtype='float32', padding='post', maxlen=200)
    
    print('Text preprocessing and embedding complete!')
    return list_

In [10]:
# def preprocess_pad(list_text):
#     df_clean=text_cleaner(X_text)
#     df_clean.reset_index(drop = True, inplace=True)
#     word2vec_transfer = api.load('glove-wiki-gigaword-100')
#     X_train_embed_2 = embedding(word2vec_transfer, df_clean['clean_text'])
#     X_train_pad = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
#     return X_train_pad

In [11]:
X_text = preprocess_pad(X_text)

Cleaning text
Embedding with word2vec
Padding sequences
Text preprocessing and embedding complete!


# Functional Model

In [12]:
def build_model():
    
    # CNN Architecture
    cnn_input = layers.Input(shape=X_image.shape[1:])

    x = layers.Conv2D(16, (4, 4), activation='relu')(cnn_input)
    x = layers.MaxPool2D(2, 2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(16, (4, 4), activation='relu')(x)
    x = layers.MaxPool2D(2, 2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(16, (4, 4), activation='relu')(x)
    x = layers.MaxPool2D(2, 2)(x)
    x = layers.BatchNormalization()(x)
    
    cnn_output = layers.Flatten()(x)
    
    # LSTM Architecture
    nlp_input =layers.Input(shape=(200,100))
    y = layers.Masking()(nlp_input)
    y = layers.BatchNormalization()(y)

    y = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=True))(y)
    y = layers.Dropout(0.5)(y)

    y = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=True))(y)
    y = layers.Dropout(0.5)(y)

    y = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=False))(y)
    nlp_output = layers.Dropout(0.5)(y)
    
    # Define NLP model and concatenate output
    combined = layers.concatenate([cnn_output, nlp_output])
    
    z = layers.Dense(64, activation='relu')(combined)
    z = layers.Dropout(0.4)(z)
    z = layers.Dense(32, activation='relu')(z)
    z = layers.Dropout(0.2)(z)
    
    final_output = layers.Dense(1, activation='sigmoid')(z)

    model = Model(inputs=[cnn_input, nlp_input],outputs=final_output)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

In [13]:
model = build_model()

In [14]:
# model.summary()

In [15]:
history = model.fit(
    (X_image, X_text),
    y,
    epochs=50,
    batch_size=16,
    validation_split=0.3,
    callbacks=callbacks.EarlyStopping(patience=5, restore_best_weights=True),
)

2022-12-07 00:45:12.711430: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4065120000 exceeds 10% of free system memory.


Epoch 1/50


KeyboardInterrupt: 